# Baseline
Setting up a more robust baseline notebook, suitable for use with all of the "Big Three" (XGBoost, CatBoost, LightGBM) libraries and on either Google Colab or the local machine.

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True
# libraries = ['xgboost', 'lightgbm', 'catboost']
libraries = ['xgboost', 'lightgbm', 'catboost']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"stacking_manual_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
#     !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

#     !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if USE_GPU: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if USE_GPU:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
# import timm

import seaborn as sns

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler #, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [39]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/oct2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [8]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Ex-Model Config

In [9]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [10]:
# if exmodel_config['scaler']:
#     scaler = exmodel_config['scaler']()
#     scaler.fit_transform()

In [11]:
train_source = datapath/'train.feather'
df = pd.read_feather(path=train_source)
df.index.name = 'id'
y_train = df.target
features = [x for x in df.columns if x != 'target']
X_train = df[features]
# X.index.name = 'id'
# y.index.name = 'id'
X = np.array(X_train)
y = np.array(y_train)

# del df, X_train, y_train


# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

exmodel_config['train_source'] = str(train_source)

In [12]:
test_source = datapath/'test.feather'
exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]

In [13]:
X_test = np.array(X_test)

## Weights and Biases Run Config

Below is the configuration for a Weights and Biases (`wandb`) run.

In [14]:
# wandb config:
wandb_config = {
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'tags': ['stacking-manual'],
    'notes': "Using best-to-date params on GBM classifiers from XGBoost, LightGBM, and CatBoost on original, unaltered dataset. Manual stacking ensemble, with two random-state versions of each architecture in first layer, then one in second, then a LogisticRegressor for third.",
}

# Training

# Hyperparameters

In [15]:
# optuna 20211004, thru 106 trials on unaltered original dataset
best_xgboost_params = {
    'n_estimators': 3878,
    'max_depth': 4,
    'learning_rate': 0.024785857161974977,
    'reg_alpha': 26.867682044658245,
    'reg_lambda': 10.839759074147148,
    'subsample': 0.8208581489835881,
    'min_child_weight': 8.829122644339664,
    'colsample_bytree': 0.906420714280384,
    'gamma': 1.472322916021486
}

# best as of 20211005, thru 65 trials on unaltered original dataset
best_lightgbm_params = {
    'n_estimators': 6631,
    'max_depth': 10,
    'learning_rate': 0.004677044539666842,
    'reg_alpha': 19.334971246299116,
    'reg_lambda': 0.024384251140153856,
    'subsample': 0.5082183652689569,
    'boosting_type': 'gbdt',
    'min_child_samples': 9,
    'num_leaves': 233,
    'colsample_bytree': 0.5008014086989773
}

# catboost 20211001 on colab with 100 trials on GPU, unaltered original dataset
best_catboost_params = {
    'iterations': 29338,
    'max_depth': 9,
    'learning_rate': 0.004769831650275205,
    'random_strength': 7,
    'od_wait': 1968,
    'reg_lambda': 28.435563240493586,
    'border_count': 162,
    'min_child_samples': 14,
    'leaf_estimation_iterations': 1
}

In [38]:
def cross_validate_model(library:str, params:dict={}, X=X, y=y, X_test=X_test, start_fold=0, 
                         exmodel_config=exmodel_config, wandb_config=wandb_config, 
                         random_state=42, wandb_tracked=True):
    """
    Function to handle model training process in the context of cross-validation -- via hold-out or via k-fold.
    If exmodel_config['cross_val_strategy'] == None, then any kfolds= input is ignored; otherwise, the number specified is used.
    
    :param kfolds: int specifying number of k-folds to use in cross-validation
    :param exmodel_config: dict containing general config including for cross-validation -- `kfold=1` implies hold-out
    """
#     if exmodel_config['kfolds'] == 1:
#         print("Proceeding with holdout")
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
#                                                           test_size=0.2, 
#                                                           random_state=SEED)                 
    
    # prepare for k-fold cross-validation; random-state here is notebook-wide, not per-model
    kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=SEED)
    
    if wandb_tracked:
        exmodel_config['library'] = library
        exmodel_config[f'{library}_params'] = str(params)
        wandb.init(
            project="202110_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )   
    
    # setup for serialization
    runpath = Path(modelpath/f"{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds/")
    (runpath).mkdir(exist_ok=True)
    
    
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
    
    # initialize a numpy.ndarray containing the fold-model's preds for test set
    test_preds = np.zeros((X_test.shape[0]))
    
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
        if fold < start_fold: # skip folds that are already trained
            continue
        else:
            print(f"FOLD {fold}")
            print("---------------------------------------------------")
            X_train, X_valid = X[train_ids], X[valid_ids]
            y_train, y_valid = y[train_ids], y[valid_ids]
    
        # define models
        if library == 'xgboost':
            model = XGBClassifier(
                booster='gbtree',
                tree_method='gpu_hist',
                random_state=random_state,
                n_jobs=-1, 
                verbosity=1, 
                objective='binary:logistic',
                **params)
            if wandb_tracked:
                model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
            else:
                model.fit(X_train, y_train)


        elif library == 'lightgbm':
            model = LGBMClassifier(
                objective='binary',
                random_state=random_state,
#                 n_jobs=-1,
#                 eval_metric='auc',
                device_type='gpu',
                max_bin=63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
                gpu_use_dp=False, # forces use of single precision rather than double for better perf, esp on consumer Nvidia chips
                **params)
            if wandb_tracked:
                model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()],)
            else:
                model.fit(X_train, y_train)

            
        elif library == 'catboost':
            model = CatBoostClassifier(
                task_type='GPU',
                silent=True,
                random_state=random_state,
                **params) 
        
            model.fit(X_train, y_train)
        
        # take the training set predictions, if desired
#         y_train_pred = model.predict_proba(X_train)[:,1]
#         train_loss = log_loss(y_train, y_train_pred)
#         train_auc = roc_auc_score(y_train, y_train_pred)
#         wandb.log({'train_loss': train_loss, 'train_auc': train_auc})

        # log the parameters, if desired
#         if exmodel_config['library'] == 'catboost':
#             print(model.get_all_params())
#             wandb.log(model.get_all_params())
#         else:
#             wandb.log(model.get_params()) # logging model parameters, trying bare-invocation rather than params: model.get_params()

        y_valid_preds = model.predict_proba(X_valid)[:,1]
        
        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)
        
        # add the fold's predictions to the model's test-set predictions (will divide later)
        test_preds += model.predict_proba(X_test)[:,1]

#         valid_loss = log_loss(y_valid, y_pred)
        # give the valid AUC score, for edification
        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
        dump(model, Path(runpath/f"{exmodel_config['library']}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for {library} model is {model_valid_auc}")
    
    # finalize test preds
    test_preds /= exmodel_config['kfolds']
    
    # save OOF preds and test-set preds
    dump(oof_preds, Path(predpath/f"{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds_rs{random_state}_oof_preds.joblib"))
    dump(test_preds, Path(predpath/f"{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds_rs{random_state}_test_preds.joblib"))
    dump(oof_y, predpath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib")
    
    if wandb_tracked:
        wandb.log({'model_valid_auc': model_valid_auc,
                   'oof_preds': oof_preds,
                   'test_preds': test_preds,
                   'model_params': str(model.get_params()),
                  })
        wandb.finish()
    return oof_preds, test_preds
        

In [17]:
# _, lightgbm_preds = cross_validate_model(library='lightgbm')

In [18]:
# _, catboost_preds = cross_validate_model(library='catboost')

In [19]:
# xgboost_oof_preds, xgboost_test_preds = cross_validate_model(library='xgboost')

# Single Submission

In [20]:
# sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [21]:
# sample_df.loc[:, 'target'] = xgboost_preds

In [22]:
# sample_df.head()

In [23]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [24]:
# sample_df.to_csv(subpath/f"{wandb_config['name']}_xgboost_{exmodel_config['kfolds']}folds_rs{42}_baseline_preds.csv", index=False)

In [25]:
# str(blender.estimators[2][1].get_all_params())
# blender.estimators[2][1]

In [26]:
# wandb.log({'leaderboard_auc': 0.81725,
# #            'catboost_params': str(best_catboost_params),
#           })

In [27]:
# wandb.finish()

In [28]:
# oof_y_pd = pd.Series(oof_y)

# Predictions

## Level One

In [29]:
oof_lv1, test_lv1 = pd.DataFrame(), pd.DataFrame() # initialize dataframes

### Generating

In [30]:
oof_lv1_lgb42, test_lv1_lgb42 = cross_validate_model(library='lightgbm', X=X, y=y, X_test=X_test, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_lightgbm_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=True
                                        )
oof_lv1['lgb42'] = oof_lv1_lgb42
test_lv1['lgb42'] = test_lv1_lgb42

wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.8572718743878947
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.8559994877532525
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.8568325308014939
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.8559842308123811
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.8565528831795393
Valid AUC score for lightgbm model is 0.8565251751685374


wandb: WARNING Serializing object of type ndarray that is 4000104 bytes


model_valid_auc,▁
model_params,{'boosting_type': 'g...
model_valid_auc,0.85653


In [31]:
oof_lv1_lgb1983, test_lv1_lgb1983 = cross_validate_model(library='lightgbm', X=X, y=y, X_test=X_test, 
                                                 wandb_config=wandb_config,
                                                 random_state=1983,
                                                 params=best_lightgbm_params,
                                                 exmodel_config=exmodel_config, 
                                                 wandb_tracked=True
                                                )
oof_lv1['lgb1983'] = oof_lv1_lgb1983
test_lv1['lgb1983'] = test_lv1_lgb1983

wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.8572161841869534
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.8559350043052648
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.8568529019126492
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.8558748673652216
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.8565261379114149
Valid AUC score for lightgbm model is 0.8564776078917813


wandb: WARNING Serializing object of type ndarray that is 4000104 bytes


model_valid_auc,▁
model_params,{'boosting_type': 'g...
model_valid_auc,0.85648


In [32]:
oof_lv1_xgb42, test_lv1_xgb42 = cross_validate_model(library='xgboost', X=X, y=y, X_test=X_test, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_xgboost_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=True
                                        )

oof_lv1['xgb42'] = oof_lv1_xgb42
test_lv1['xgb42'] = test_lv1_xgb42

wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FOLD 0
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


Valid AUC for fold 0 is 0.8572554115376164
FOLD 1
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


...

Valid AUC for fold 3 is 0.8560833380957398
FOLD 4
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


Valid AUC for fold 4 is 0.8567086183230934
Valid AUC score for xgboost model is 0.8566841128860819


wandb: WARNING Serializing object of type ndarray that is 4000104 bytes


model_valid_auc,▁
model_params,{'objective': 'binar...
model_valid_auc,0.85668


In [33]:
oof_lv1_xgb1983, test_lv1_xgb1983 = cross_validate_model(library='xgboost', X=X, y=y, X_test=X_test, 
                                                 wandb_config=wandb_config,
                                                 random_state=1983,
                                                 params=best_xgboost_params,
                                                 exmodel_config=exmodel_config, 
                                                 wandb_tracked=True
                                                )
oof_lv1['xgb1983'] = oof_lv1_xgb1983
test_lv1['xgb1983'] = test_lv1_xgb1983

wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FOLD 0
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


Valid AUC for fold 0 is 0.8573146071386168
FOLD 1
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


...

Valid AUC for fold 3 is 0.8560595010136646
FOLD 4
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


Valid AUC for fold 4 is 0.8566916357893442
Valid AUC score for xgboost model is 0.8566807985789633


wandb: WARNING Serializing object of type ndarray that is 4000104 bytes


model_valid_auc,▁
model_params,{'objective': 'binar...
model_valid_auc,0.85668


In [34]:
oof_lv1_cat42, test_lv1_cat42 = cross_validate_model(library='catboost', X=X, y=y, X_test=X_test, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_catboost_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=True
                                        )
oof_lv1['cat42'] = oof_lv1_cat42
test_lv1['cat42'] = test_lv1_cat42

wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.8575766033930445
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.8564289538689146
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.8574776355547533
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.856314002656122
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.8570333546927973
Valid AUC score for catboost model is 0.8569640294774554


wandb: WARNING Serializing object of type ndarray that is 4000104 bytes


model_valid_auc,▁
model_params,{'iterations': 29338...
model_valid_auc,0.85696


In [54]:
oof_lv1_cat1983, test_lv1_cat1983 = cross_validate_model(library='catboost', X=X, y=y, X_test=X_test, 
                                                 wandb_config=wandb_config,
                                                 random_state=1983,
                                                 params=best_catboost_params,
                                                 exmodel_config=exmodel_config, 
                                                 wandb_tracked=False
                                                )
oof_lv1['cat1983'] = oof_lv1_cat1983
test_lv1['cat1983'] = test_lv1_cat1983

FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.8575796749930964
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.8564173360521683
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.8575358682866415
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.8563227427678826
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.8570098742696105
Valid AUC score for catboost model is 0.856970844007867


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Loading Sets of Predictions

In [ ]:
# oof_lv1, test_lv1 = pd.DataFrame(), pd.DataFrame()
# preds_path = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/preds/')

In [ ]:
# oof_lv1['xgb42'] = load(predpath/'stacking_manual_20211005_085253_xgboost_5folds_rs42_oof_preds.joblib')
# test_lv1['xgb42'] = load(preds_path/'stacking_manual_20210925_212129_xgboost_5folds_rs42_test_preds.joblib')

In [ ]:
# oof_lv_xgb42_y = load(predpath/'stacking_manual_20211005_085253_xgboost_5folds_rs42_oof_y.joblib')

In [ ]:
# roc_auc_score(y_true=oof_lv_xgb42_y, y_score=oof_lv1['xgb42'])

In [ ]:
# oof_lv1['xgb1983'] = load(preds_path/'validAUC_0.8146252172737458_stacking_manual_20210926_211701_xgboost_5folds_rs1983_oof_preds.joblib')
# test_lv1['xgb1983'] = load(preds_path/'stacking_manual_20210926_211701_xgboost_5folds_rs1983_test_preds.joblib')

In [ ]:
# oof_lv1['lgb42'] = load(preds_path/'validAUC_0.8156810521798477_stacking_manual_20210925_212129_lightgbm_5folds_rs42_oof_preds.joblib')
# test_lv1['lgb42'] = load(preds_path/'stacking_manual_20210925_212129_lightgbm_5folds_rs42_test_preds.joblib')

In [ ]:
# oof_lv1['lgb1983'] = load(preds_path/'validAUC_0.8156503194185875_stacking_manual_20210925_212129_lightgbm_5folds_rs1983_oof_preds.joblib')
# test_lv1['lgb1983'] = load(preds_path/'stacking_manual_20210925_212129_lightgbm_5folds_rs1983_test_preds.joblib')

In [ ]:
# oof_lv1['cat42'] = load(preds_path/'validAUC_0.8116727090290558_stacking_manual_20210925_212129_catboost_5folds_rs42_oof_preds.joblib')
# test_lv1['cat42'] = load(preds_path/'stacking_manual_20210925_212129_catboost_5folds_rs42_test_preds.joblib')

In [ ]:
# oof_lv1['cat1983'] = load(predpath/'stacking_manual_20211005_085253_catboost_5folds_rs1983_oof_preds.joblib')
# oof_cat1983_y = load(predpath/'stacking_manual_20211005_085253_catboost_5folds_rs1983_oof_y.joblib')
# roc_auc_score(y_true=oof_cat1983_y, y_score=oof_lv1['cat1983'])
# test_lv1['cat1983'] = load(preds_path/'stacking_manual_20210925_212129_catboost_5folds_rs1983_test_preds.joblib')

In [ ]:
# oof_cat42_y = load(predpath/'stacking_manual_20211005_085253_catboost_5folds_rs42_oof_y.joblib')

In [ ]:
# oof_cat42_y == oof_lv_xgb42_y

In [ ]:
# oof_lv1.iloc[:20, :]

In [ ]:
# oof_y_pd.iloc[:20]

- Why is it that the random seed seems far more important than the model type in making a prediction???

### Serialization

In [40]:
oof_lv1.to_csv('oof_lv1.csv', index=False)
test_lv1.to_csv('test_lv1.csv', index=False)

In [41]:
oof_lv1.to_feather(predpath/f"{wandb_config['name']}_oof_lv1.feather")
test_lv1.to_feather(predpath/f"{wandb_config['name']}_test_lv1.feather")

### Lv1 Finalization

In [42]:
oof_lv1 = pd.read_feather(predpath/f"{wandb_config['name']}_oof_lv1.feather")#, columns=[str(x) for x in range()])
test_lv1 = pd.read_feather(predpath/f"{wandb_config['name']}_oof_lv1.feather")
oof_y = load(predpath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib")

# oof_lv1.index.name = 'id'
# test_lv1.index.name = 'id'
# oof_y.index.name = 'id'
oof_lv1 = np.array(oof_lv1)
test_lv1 = np.array(test_lv1)
oof_y = np.array(oof_y)

## Level Two

In [43]:
oof_lv2, test_lv2 = pd.DataFrame(), pd.DataFrame()

In [46]:
oof_lv2_xgb42, test_lv2_xgb42 = cross_validate_model(library='xgboost', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_xgboost_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=False
                                        )

dump(oof_lv2_xgb42, predpath/f"{wandb_config['name']}_oof_lv2_xgboost42_preds.joblib")
dump(test_lv2_xgb42, predpath/f"{wandb_config['name']}_test_lv2_xgboost42_preds.joblib")



FOLD 0
---------------------------------------------------
[05:54:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Valid AUC for fold 0 is 0.8567673974385948
FOLD 1
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:54:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8572241681755651
FOLD 2
---------------------------------------------------


...

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:54:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 3 is 0.8569590636482101
FOLD 4
---------------------------------------------------


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:54:33] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 4 is 0.8579641095637536
Valid AUC score for xgboost model is 0.8570550391170862


['/home/sf/code/kaggle/tabular_playgrounds/oct2021/preds/stacking_manual_20211005_205933_test_lv2_xgboost42_preds.joblib']

In [ ]:
# oof_xgb_f0_rs1983 = load('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/models/stacking_manual_20210926_211701_xgboost_5folds/xgboost_fold0_model.joblib')
# oof_xgb_f0_rs42 = load('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/models/stacking_manual_20210925_212129_xgboost_5folds/xgboost_fold0_model.joblib')

In [47]:
oof_lv2_cat42, test_lv2_cat42 = cross_validate_model(library='catboost', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_catboost_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=False
                                        )

FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.8561112140085647
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.8566038972798236
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.8558367995062315
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.8563649950153741
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.8573670243744347
Valid AUC score for catboost model is 0.8564461628301947


In [49]:
dump(oof_lv2_cat42, predpath/f"{wandb_config['name']}_oof_lv2_catboost42_preds.joblib")
dump(test_lv2_cat42, predpath/f"{wandb_config['name']}_test_lv2_catboost42_preds.jobli")

['/home/sf/code/kaggle/tabular_playgrounds/oct2021/preds/stacking_manual_20211005_205933_test_lv2_catboost42_preds.jobli']

In [50]:

oof_lv2_lgb42, test_lv2_lgb42 = cross_validate_model(library='lightgbm', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=best_lightgbm_params,
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=False
                                        )

FOLD 0
---------------------------------------------------


LightGBMError: Check failed: (best_split_info.left_count) > (0) at /home/sf/Software/LightGBM/src/treelearner/serial_tree_learner.cpp, line 653 .


In [ ]:
dump(oof_lv2_lgb42, predpath/f"{wandb_config['name']}_oof_lv2_lightgbm42_preds.joblib")
dump(test_lv2_lgb42, predpath/f"{wandb_config['name']}_test_lv2_lightgbm42_preds.joblib")

In [ ]:
oof_lv2['xgboost'] = oof_lv2_xgb42
oof_lv2['catboost'] = oof_lv2_cat42
oof_lv2['lightgbm'] = oof_lv2_lgb42

test_lv2['xgboost'] = test_lv2_xgb42
test_lv2['catboost'] = test_lv2_cat42
test_lv2['lightgbm'] = test_lv2_lgb42

In [ ]:
oof_lv2.head()

In [ ]:
oof_lv1_df = pd.read_feather(predpath/'oof_lv1.feather')

In [ ]:
oof_lv1_df.head()

In [ ]:
oof_lv2_full = oof_lv2.join(oof_lv1_df)

In [ ]:
test_lv2_full = test_lv2.join(pd.read_feather(preds_path/'test_lv1.feather'))

In [ ]:
oof_lv2_full.head()

In [ ]:
test_lv2_full.head()

In [ ]:
oof_lv2_np = oof_lv2_full.to_numpy()
test_lv2_np = test_lv2_full.to_numpy()

In [ ]:
oof_y_np = oof_y

## Level Three (Logistic Regression)

In [ ]:
from sklearn import model_selection

In [ ]:
kfolds = model_selection.StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
oof_preds, oof_y = [], []

In [ ]:
test_preds = np.zeros((X_test.shape[0]))

In [ ]:
X = oof_lv2_np
y = oof_y_np
X_test = test_lv2_np

In [ ]:
library = 'sklearn (LogisticRegressor(max_iter=1000))'
exmodel_config['library'] = library
# wandb.init(
#     project="202110_Kaggle_tabular_playground",
#     save_code=True,
#     tags=wandb_config['tags'],
#     name=wandb_config['name'],
#     notes=wandb_config['notes'],
#     config=exmodel_config
# )   

# # prepare for k-fold cross-validation
# kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=random_state)

# setup for serialization
# model_path = Path(datapath/f"models/{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds/")
# (model_path).mkdir(exist_ok=True)

In [ ]:
for fold, (train_idx, valid_idx) in enumerate(kfolds.split(X,y)):
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    
    print(f"FOLD {fold}")
    print("---------------------")
    
    model = LogisticRegression(max_iter=1000)
    
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_valid)[:,1]
    
    oof_preds.extend(preds)
    oof_y.extend(y_valid)
    
    test_preds += model.predict_proba(X_test)[:,1]
    
    valid_auc = roc_auc_score(y_valid, preds)
    print(f"ROC AUC of fold {fold} is {valid_auc}")
    
#     dump(preds, /'lv_3)

valid_auc_total = roc_auc_score(oof_y, oof_preds)
print(f"Overall ROC_AUC is {valid_auc_total}")

dump(oof_preds, predpath/'oof_lv3_preds.joblib')
dump(oof_y, predpath/'oof_lv3_y.joblib')

test_preds /= 5

dump(test_preds, predpath/'test_lv3_preds.joblib')

In [ ]:
# X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_imputed-Median-wIndicators_StandardScaled.feather')
# X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')

## Prediction Generation

In [ ]:
# preds_path = Path(datapath/"preds/")

# blender_preds = blender.predict_proba(X_test_imputed_scaled)[:,1]
# dump(blender_preds, preds_path/f"{config_run['name']}_stack.joblib")

# Ensemble Submission

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = test_preds

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-ensemble_{exmodel_config['kfolds']}folds_rs{42}_baseline_preds.csv", index=False)

In [56]:
wandb.log({'leaderboard_auc': ,
#            'catboost_params': str(best_catboost_params),
          })

In [57]:
wandb.finish()

blender-cv,5
blender-final_estimator,LogisticRegression(m...
blender-stack_mdethod,predict_proba
leaderboard_auc,0.81725
lightgbm_params,{'boosting_type': 'g...
test_set,/media/sf/easystore/...
train_auc,0.8414
train_loss,0.494
xgboost_params,{'objective': 'binar...


blender-cv,▁
leaderboard_auc,▁
train_auc,▁
train_loss,▁
